In [4]:
from pyspark.sql import functions as F, SparkSession

In [5]:
# Create a Spark Session
spark = (
    SparkSession.builder.appName("consumer model")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.execturo.memory", "2g")
    .getOrCreate()
)

In [6]:
consumer_info = spark.read.parquet('../data/curated/consumer_info.parquet')
transaction_records = spark.read.parquet('../data/curated/transaction_records.parquet')
fraudulent_consumer_rate = spark.read.parquet('../data/curated/consumer_fraud_rate.parquet')

In [7]:
consumer_transaction_records = transaction_records.join(consumer_info, on="consumer_id", how="inner")

In [20]:
fraudulent_consumer_rate.count()

34864

In [8]:
consumer_transaction_records.join(fraudulent_consumer_rate, on=['consumer_id', 'order_datetime'], how='inner').count()

80560

In [27]:
consumer_transaction_records.join(fraudulent_consumer_rate, on=['consumer_id', 'order_datetime'], how='inner')

consumer_id,order_datetime,merchant_abn,dollar_value,order_id,name,gender,state,postcode,fraud_probability
838663,2021-11-26,43725628716,2684.361417897887,52133cb6-4950-44c...,Adam Velazquez,Male,NSW,2403,9.241197906604826
838663,2021-11-26,45629217853,0.19607762574988435,e5b1880d-fb88-4fd...,Adam Velazquez,Male,NSW,2403,9.241197906604826
838663,2021-11-26,73499119023,9.260984119033456,a576a634-dcd0-445...,Adam Velazquez,Male,NSW,2403,9.241197906604826
640907,2021-11-26,30623214058,111.91165017514544,362fc544-e9c0-449...,Terry Martin,Male,SA,5168,15.645332445256162
640907,2021-11-26,79417999332,54.89560911946132,3c246238-a15f-429...,Terry Martin,Male,SA,5168,15.645332445256162
640907,2021-11-26,18305506006,5.198633525613379,e4cb215d-26d8-4e4...,Terry Martin,Male,SA,5168,15.645332445256162
640907,2021-11-26,96244711717,52.48791119040359,3a3c80b1-be38-447...,Terry Martin,Male,SA,5168,15.645332445256162
640907,2021-11-26,34967436738,4298.699244530939,2630e79b-6148-4b3...,Terry Martin,Male,SA,5168,15.645332445256162
640907,2021-11-26,16248082282,361.5545523024534,8f448a5a-fe7c-4fc...,Terry Martin,Male,SA,5168,15.645332445256162
192322,2021-11-26,45629217853,26.34821606933028,99c767d9-811b-4cd...,Aaron Hall,Male,WA,6396,19.10895425125948


In [9]:
consumer_transaction_records.columns

['consumer_id',
 'merchant_abn',
 'dollar_value',
 'order_id',
 'order_datetime',
 'name',
 'gender',
 'state',
 'postcode']

In [10]:
consumer_transaction_records.select("dollar_value").summary()

summary,dollar_value
count,14195505
mean,166.22895390891753
stddev,517.8505602612809
min,9.756658099412162E-8
25%,26.128899475514228
50%,62.22903376686649
75%,150.41282294992547
max,105193.88578925544
